# Assignment Week-3 Part-3

## - Nishant Kumar

In [1]:
# Installing libraries
!pip install beautifulsoup4
!pip install folium
!pip install geocoder
!pip install lxml

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
# importing libraries

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests as re
import folium
import geocoder
from sklearn.cluster import KMeans
import matplotlib.cm as cm

## 1) Scraping Data from wikipedia website

In [3]:
# Creating dataframe columns
canada_neigh = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
canada_neigh

,PostalCode,Borough,Neighborhood


In [4]:
# creating requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = re.get(url).text

In [5]:
# creating soup using beautiful soup
soup = BeautifulSoup(request, 'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1e672fe4-7de0-4909-827e-860898a32f71","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protected pa

In [6]:
# selecting target html tag and adding the data into the dataframe
soup_list = soup.select('tbody > tr > td')
for soup in soup_list:
    if len(soup.text.replace('\n','').replace('/',',').replace(')','').split('(')) > 1:
        postal_code = soup.text.replace('\n','')[0:3]
        borough = soup.text.replace('\n','').replace('/',',').replace(')','').split('(')[0][3:]
        neigh = soup.text.replace('\n','').replace('/',',').replace(')','').split('(')[1]
        canada_neigh = canada_neigh.append({
            'PostalCode' : postal_code,
            'Borough' : borough,
            'Neighborhood' : neigh
        }, ignore_index=True )
canada_neigh

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


## 2) Finding Latitude and Longitude of postal code and adding data to dataframe

In [7]:
# function to generate latitude and longitude using geocoder library
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords

In [8]:
for index, row in canada_neigh.iterrows():
    postal_code = row['PostalCode']
    lat_long = get_latilong(postal_code)
    canada_neigh.loc[index, 'Latitude'] = lat_long[0]
    canada_neigh.loc[index, 'Longitude'] = lat_long[1]
canada_neigh

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.63278,-79.48945


## 3) Exploring and clustering borough containing word "Toronto"

In [9]:
# Extracting Borough containing term "Toronto"
toronto_neigh = canada_neigh[canada_neigh['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_neigh

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
5,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
6,M6G,Downtown Toronto,Christie,43.66869,-79.42071
7,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.64970,-79.38258
8,M6H,West Toronto,"Dufferin , Dovercourt Village",43.66505,-79.43891
9,M4J,East YorkEast Toronto,The Danforth East,43.68811,-79.33418


In [10]:
# Map using folium library
latitude = 43.6532
longitude = -79.3832
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, label in zip(toronto_neigh['Latitude'], toronto_neigh['Longitude'], toronto_neigh['Neighborhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
toronto_map

In [11]:
# Defining Foursquare Credentials and Version
CLIENT_ID = '33ZY1DLZRO0F2X5H3OOIXBIS1XFY4DE54QFO2BUIOEBNDJNY' # your Foursquare ID
CLIENT_SECRET = 'O2ZC2KVZU1WG5GNN2TTFXGKAPUJ3PE3YTA250DGH3FTFKZ4M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
toronto_df_new = toronto_neigh.copy()

Your credentails:
CLIENT_ID: 33ZY1DLZRO0F2X5H3OOIXBIS1XFY4DE54QFO2BUIOEBNDJNY
CLIENT_SECRET:O2ZC2KVZU1WG5GNN2TTFXGKAPUJ3PE3YTA250DGH3FTFKZ4M


In [12]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, radius, LIMIT)
    
    results = re.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((post, borough, neighborhood, lat, long,
                       venue['venue']['name'],
                       venue['venue']['location']['lat'],
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

In [13]:
# Data frame of venues
venues_df = pd.DataFrame(venues, columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory'])
venues_df.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1704, 226)


,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
toronto_grouped.head()

,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,M4J,East YorkEast Toronto,The Danforth East,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,M4K,East Toronto,"The Danforth West , Riverdale",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,M4L,East Toronto,"India Bazaar , The Beaches West",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,M4M,East Toronto,Studio District,0.0,0.0,0.039216,0.019608,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.019608,0.0,0.0,0.0,0.0


In [16]:
# dataframe of top 10 venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Deli / Bodega,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4J,East YorkEast Toronto,The Danforth East,Park,Convenience Store,Fabric Shop,Intersection,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,M4K,East Toronto,"The Danforth West , Riverdale",Bus Line,Grocery Store,Coffee Shop,Ice Cream Shop,Discount Store,Business Service,Park,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
3,M4L,East Toronto,"India Bazaar , The Beaches West",Sandwich Place,Fast Food Restaurant,Pub,Playground,Sushi Restaurant,Italian Restaurant,Restaurant,Movie Theater,Dry Cleaner,Ice Cream Shop
4,M4M,East Toronto,Studio District,Italian Restaurant,Diner,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar,Bakery,Pizza Place,Gastropub,Coffee Shop


### Clustering using K-Means 

In [17]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 1, 2, 0, 0, 1, 3], dtype=int32)

In [18]:
toronto_final = neighborhoods_venues_sorted.copy()

In [19]:
toronto_final['Latitude'] = toronto_neigh['Latitude']
toronto_final['Longitude'] = toronto_neigh['Longitude']

In [20]:
toronto_final['K Means Label'] = kmeans.labels_

In [21]:
toronto_final

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,K Means Label
0,M4E,East Toronto,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Deli / Bodega,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,43.65512,-79.36264,1
1,M4J,East YorkEast Toronto,The Danforth East,Park,Convenience Store,Fabric Shop,Intersection,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,43.65739,-79.37804,3
2,M4K,East Toronto,"The Danforth West , Riverdale",Bus Line,Grocery Store,Coffee Shop,Ice Cream Shop,Discount Store,Business Service,Park,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,43.65215,-79.37587,1
3,M4L,East Toronto,"India Bazaar , The Beaches West",Sandwich Place,Fast Food Restaurant,Pub,Playground,Sushi Restaurant,Italian Restaurant,Restaurant,Movie Theater,Dry Cleaner,Ice Cream Shop,43.67709,-79.29547,1
4,M4M,East Toronto,Studio District,Italian Restaurant,Diner,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar,Bakery,Pizza Place,Gastropub,Coffee Shop,43.64536,-79.37306,1
5,M4N,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,43.65609,-79.38493,2
6,M4P,Central Toronto,Davisville North,Department Store,Gym / Fitness Center,Park,Convenience Store,Hotel,Playground,Food & Drink Shop,Breakfast Spot,Falafel Restaurant,Ethiopian Restaurant,43.66869,-79.42071,0
7,M4R,Central Toronto,North Toronto West,Gym Pool,Park,Playground,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,43.64970,-79.38258,0
8,M4S,Central Toronto,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Coffee Shop,Café,Indoor Play Area,Farmers Market,Skating Rink,Fast Food Restaurant,43.66505,-79.43891,1
9,M4T,Central Toronto,"Moore Park , Summerhill East",Gym,Park,Convenience Store,Restaurant,Yoga Studio,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,43.68811,-79.33418,3


### Visualizing the optained model  

In [22]:
# Map using folium library
latitude = 43.6172010546183
longitude = -79.3832
toronot_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

rainbow = ['blue', 'red', 'yellow', 'green', 'violet', 'black']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighborhoods'], toronto_final['K Means Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(toronot_clusters)
       
toronot_clusters

In [23]:
toronto_final[toronto_final['K Means Label'] == 1]

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude,K Means Label
0,M4E,East Toronto,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Deli / Bodega,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,43.65512,-79.36264,1
2,M4K,East Toronto,"The Danforth West , Riverdale",Bus Line,Grocery Store,Coffee Shop,Ice Cream Shop,Discount Store,Business Service,Park,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,43.65215,-79.37587,1
3,M4L,East Toronto,"India Bazaar , The Beaches West",Sandwich Place,Fast Food Restaurant,Pub,Playground,Sushi Restaurant,Italian Restaurant,Restaurant,Movie Theater,Dry Cleaner,Ice Cream Shop,43.67709,-79.29547,1
4,M4M,East Toronto,Studio District,Italian Restaurant,Diner,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar,Bakery,Pizza Place,Gastropub,Coffee Shop,43.64536,-79.37306,1
8,M4S,Central Toronto,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Coffee Shop,Café,Indoor Play Area,Farmers Market,Skating Rink,Fast Food Restaurant,43.66505,-79.43891,1
10,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Escape Room,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,43.64285,-79.38076,1
12,M4X,Downtown Toronto,"St. James Town , Cabbagetown",Park,Coffee Shop,Pizza Place,Café,Bakery,Pub,Restaurant,Italian Restaurant,Pet Store,Gourmet Shop,43.68375,-79.35512,1
13,M4Y,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Café,Mediterranean Restaurant,Pizza Place,Pub,Dance Studio,Men's Store,43.64710,-79.38153,1
14,M5A,Downtown Toronto,"Regent Park , Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Electronics Store,Italian Restaurant,Bakery,Pub,Thai Restaurant,Theater,Food Truck,43.63941,-79.42676,1
15,M5B,Downtown Toronto,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Hotel,Sandwich Place,Theater,Bubble Tea Shop,Italian Restaurant,43.66797,-79.31467,1


It can be clearly observed that most of the neighborhood falls into cluster 1, which commonly have Bar, Cafe, Hotel etc